In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion -- From the website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader('https://en.wikipedia.org/wiki/Valorant')
loader

In [8]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Valorant', 'title': 'Valorant - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nValorant - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nGameplay\n\n\n\n\

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

In [10]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Valorant', 'title': 'Valorant - Wikipedia', 'language': 'en'}, page_content='Valorant - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nGameplay\n\n\n\n\nToggle 

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [12]:
from langchain_community.vectorstores import FAISS

vectorStoresdb = FAISS.from_documents(documents, embeddings)

In [13]:
vectorStoresdb

In [15]:
## Query from Vector store DB

query="It currently offers how many agents to choose from?"
result = vectorStoresdb.similarity_search(query)
result[0].page_content

'Agents\nThere are a large variety of playable agents available in the game. Agents are divided into 4 roles: Duelists, Sentinels, Initiators, and Controllers. Each agent has a different role which indicates how the agent is usually played.'

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [18]:
## Reterival Chain, Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate   


prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context::
    <context>
    {context}
    </context>

"""
)




document_chain = create_stuff_documents_chain(llm,prompt)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context::\n    <context>\n    {context}\n    </context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021EA8186F80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021EA8178520>, root_client=<openai.OpenAI object at 0x0000021EA93D3B80>, root_async_client=<openai.AsyncOpenAI object at 0x0000021EA8186F50>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name':

In [20]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"It currently offers how many agents to choose from?",
    "context":[Document(page_content="It currently offers 24 agents to choose from.")]
})

'Based on the provided context, there are currently 24 agents available to choose from.'

In [22]:
retriver = vectorStoresdb.as_retriever()

from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)



In [23]:
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021EEA636D40>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context::\n    <context>\n    {context}\n    </context>\n\n'), additional_kwargs={})])
    

In [24]:
## Get the response from LLM

response = retrival_chain.invoke({"input":"It currently offers 24 agents to choose from."})
response['answer']

'What is the Replication gamemode in the game, and when was it removed?\n\nReplication is a gamemode that went live on May 11, 2021. In this mode, during agent selection, players vote on which agent they would like to play as. After voting, the game randomly selects one of the agents, and the entire team plays as that agent, even if not all players have unlocked it. It is a best of nine rounds, with players switching sides after the fourth round. Players can purchase guns and shields with a set number of credits, and abilities are pre-bought. Weapons and shields are reset every round. This gamemode was removed in patch 7.0.'